In [0]:
%scala

import java.io.File
import java.nio.file.{Files, Path, Paths, StandardCopyOption}
import java.nio.file.attribute.PosixFilePermissions
import java.text.SimpleDateFormat
import java.util.{Date, Properties}

import scala.sys.process._

import org.apache.spark.SparkContext

import java.io.File
import java.nio.file.{Files, Path, Paths, StandardCopyOption}
import java.nio.file.attribute.PosixFilePermissions
import java.text.SimpleDateFormat
import java.util.{Date, Properties}
import scala.sys.process._
import org.apache.spark.SparkContext

In [0]:
%scala

val publicKey = "cat /home/ubuntu/.ssh/id_rsa.pub".!!

def addAuthorizedPublicKey(key: String): Unit = {
  val fw = new java.io.FileWriter("/home/ubuntu/.ssh/authorized_keys", /* append */ true)
  fw.write("\n" + key)
  fw.close()
}

/**
 * Inject key into executors so that the driver can ssh into them.
 */

val numExecutors = sc.getExecutorMemoryStatus.keys.size
sc.parallelize(0 until numExecutors, numExecutors).foreach { i =>
  addAuthorizedPublicKey(publicKey)
}
addAuthorizedPublicKey(publicKey)

publicKey: String =
"ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCjAnYtDQf+sEHJSKn3PNU0A0o8auMuKpcKo9lvKbx0ApAPxJ0DAVjczOhZ/is3mJeDiQdH0CdU8tggb+DYvNw4BXlSgbv7v0Lx0SUvxFOFRycCLDAo2Jk9tQEChwW0BtgD1Xq5Jld1oJ8+Y10YwfAoKFGFJPD+zS4ONkpc3c9XtbIY1a76qKJK/FR+4Tz9EWqB8wckE5tCU+dpa3DhQfyt3QSHk4PDlgcTWBCqXObKmLOaxu6309NkRMgR705X06qIx3gTqBnd7XE6tl07E629S7wiDQrUflfvnDHUAxTQwN4kRmJmIh+AWTEq/LXctVatHaf559BlzDodVNTpAQs1 root@1014-132551-ywrn3zez-10-44-235-163
"
addAuthorizedPublicKey: (key: String)Unit
numExecutors: Int = 3

In [0]:
%scala

val workers: List[String] = {
  val executors = sc.getExecutorMemoryStatus.keys.map(_.split(":").head).map { ip =>
    if (ip.startsWith("ip")) ip.stripPrefix("ip-").split('.').head.replace("-", ".")
    else ip
  }.toSet
  println("Executors = " + executors)
  executors.toList
}
println(s"Workers = " + workers)


def writeFile(path: String, contents: String, append: Boolean = false): Unit = {
  val fw = new java.io.FileWriter(path, append)
  fw.write(contents)
  fw.close()
}

def addHostfile(key: String): Unit = {
  writeFile("/home/ubuntu/.ssh/hostfile",key + "\n" , true)
}

workers.foreach { ip =>
  addHostfile(ip)
}

Executors = Set(10.44.233.230, 10.44.235.163, 10.44.227.112)
Workers = List(10.44.233.230, 10.44.235.163, 10.44.227.112)
workers: List[String] = List(10.44.233.230, 10.44.235.163, 10.44.227.112)
writeFile: (path: String, contents: String, append: Boolean)Unit
addHostfile: (key: String)Unit

In [0]:
%scala


def createIpyprofile(logStdout: Boolean = true): String = {
  
  
  val outBuffer = new collection.mutable.ArrayBuffer[String]()
  val logger = ProcessLogger(line => outBuffer += line, println(_))
  
  val exitCode = 
    Seq("ipython", "profile", "create", "--parallel", "--profile=mpi", "--profile-dir=~/.ipython/profile_mpi") ! logger
  if (logStdout) {
    outBuffer.foreach(println)
  }
  if (exitCode != 0) {
    println(s"FAILED: on host: ")
    sys.error("Command failed")
  }
  println(s"SUCCESS: on host: ")
  outBuffer.mkString("\n")
}

/**
 * Create Ipython profile on worker nodes
 */

val numExecutors = sc.getExecutorMemoryStatus.keys.size
sc.parallelize(0 until numExecutors, numExecutors).foreach { i =>
  createIpyprofile()
}

createIpyprofile: (logStdout: Boolean)String
numExecutors: Int = 3

In [0]:
%scala


def createSSHfolder(logStdout: Boolean = true): String = {
  
  
  val outBuffer = new collection.mutable.ArrayBuffer[String]()
  val logger = ProcessLogger(line => outBuffer += line, println(_))
  
  val exitCode = 
    Seq("mkdir", "-p", "/root/.ssh") ! logger
  if (logStdout) {
    outBuffer.foreach(println)
  }
  if (exitCode != 0) {
    println(s"FAILED: on host: ")
    sys.error("Command failed")
  }
  println(s"SUCCESS: on host: ")
  outBuffer.mkString("\n")
}

/**
 * Create SSH folder on worker nodes
 */

val numExecutors = sc.getExecutorMemoryStatus.keys.size
sc.parallelize(0 until numExecutors, numExecutors).foreach { i =>
  createSSHfolder()
}

createSSHfolder: (logStdout: Boolean)String
numExecutors: Int = 3

In [0]:
%scala

val publicKey = "cat /root/.ssh/id_rsa.pub".!!

def addAuthorizedPublicKey(key: String): Unit = {
  val fw = new java.io.FileWriter("/root/.ssh/authorized_keys", /* append */ true)
  fw.write("\n" + key)
  fw.close()
}


/**
 * Inject key into executors so that the driver can ssh into them.
 */

val numExecutors = sc.getExecutorMemoryStatus.keys.size
sc.parallelize(0 until numExecutors, numExecutors).foreach { i =>
  addAuthorizedPublicKey(publicKey)
}
addAuthorizedPublicKey(publicKey)

publicKey: String =
"ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC/m99pFCWf0sW8MLqHD/37QoGPmyR+LLD3j+AEvr6ymsCgdyez10mN1WVcVZ43Ouf9sbVVkgYVpr8g9Pb1LqGJzB5nQ7Yh3x3lFOzwX3L9AgZeRd2RWyJOfRs7mGlhwwyWTZK3rApmF7R6cAwSErG1lfwnsboYeBi+UQtGs+V0kqKlIhAQj4mpiP3XElRad1shAfCsVYbmBAeEjfYOaaO1ruMQQuhf8zAYQk+je95Mzys5vPJceoeP2j4+SX8gMkIXK3/4zwBXBOK1u7AWmc3PfzAzXn+AspsEfz2D0RqSEGYeS8Gn0nezE65lSmoV6V6GLqoP+HG2OsW4vS6oBNaP root@1014-132551-ywrn3zez-10-44-235-163
"
addAuthorizedPublicKey: (key: String)Unit
numExecutors: Int = 3

In [0]:
%scala


def startSSHserver(logStdout: Boolean = true): String = {
  
  
  val outBuffer = new collection.mutable.ArrayBuffer[String]()
  val logger = ProcessLogger(line => outBuffer += line, println(_))
  
  val exitCode = 
    Seq("sudo", "service", "ssh", "start") ! logger
  if (logStdout) {
    outBuffer.foreach(println)
  }
  if (exitCode != 0) {
    println(s"FAILED: on host: ")
    sys.error("Command failed")
  }
  println(s"SUCCESS: on host: ")
  outBuffer.mkString("\n")
}

/**
 * Start SSH server on worker nodes
 */

val numExecutors = sc.getExecutorMemoryStatus.keys.size
sc.parallelize(0 until numExecutors, numExecutors).foreach { i =>
  startSSHserver()
}

startSSHserver: (logStdout: Boolean)String
numExecutors: Int = 3

In [0]:
%scala

/**
 * Ssh into the given `host` and execute `command`.
 */
def ssh(host: String, logStdout: Boolean = true): String = {
  println("ssh'ing onto host - " + host)
  val outBuffer = new collection.mutable.ArrayBuffer[String]()
  val logger = ProcessLogger(line => outBuffer += line, println(_))

  val exitCode = 
    Seq("ssh", "-o", "StrictHostKeyChecking=no", "-p", "22", "-i", "/home/ubuntu/.ssh/id_rsa", s"ubuntu@$host") ! logger
  if (logStdout) {
    outBuffer.foreach(println)
  }
  if (exitCode != 0) {
    println(s"FAILED: on host: $host")
    sys.error("Command failed")
  }
  println(s"SUCCESS: on host: $host")
  outBuffer.mkString("\n")
}

workers.foreach {ip =>
  ssh(ip)
}

ssh'ing onto host - 10.44.233.230
Pseudo-terminal will not be allocated because stdin is not a terminal.
Warning: Permanently added '10.44.233.230' (ECDSA) to the list of known hosts.
Welcome to Ubuntu 18.04.5 LTS (GNU/Linux 5.4.0-1055-aws x86_64)

 * Documentation: https://help.ubuntu.com
 * Management: https://landscape.canonical.com
 * Support: https://ubuntu.com/advantage
This system has been minimized by removing packages and content that are
not required on a system that users do not log into.

To restore this content, you can run the 'unminimize' command.

The programs included with the Ubuntu system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Ubuntu comes with ABSOLUTELY NO WARRANTY, to the extent permitted by
applicable law.

SUCCESS: on host: 10.44.233.230
ssh'ing onto host - 10.44.235.163
Pseudo-terminal will not be allocated because stdin is not a terminal.
Warning: Permanently added '10.44.235.163' (ECDSA) to the list of known hosts.
Welcome to Ubuntu 18.04.5 LTS (GNU/Linux 5.4.0-1055-aws x86_64)

 * Documentation: https://help.ubuntu.com
 * Management: https://landscape.canonical.com
 * Support: https://ubuntu.com/advantage
This system has been minimized by removing packages and content that are
not required on a system that users do not log into.

To restore this content, you can run the 'unminimize' command.

The programs included with the Ubuntu system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Ubuntu comes with ABSOLUTELY NO WARRANTY, to the extent permitted by
applicable law.

SUCCESS: on host: 10.44.235.163
ssh'ing onto host - 10.44.227.112
Pseudo-terminal will not be allocated because stdin is not a terminal.
Warning: Permanently added '10.44.227.112' (ECDSA) to the list of known hosts.
Welcome to Ubuntu 18.04.5 LTS (GNU/Linux 5.4.0-1055-aws x86_64)

 * Documentation: https://help.ubuntu.com
 * Management: https://landscape.canonical.com
 * Support: https://ubuntu.com/advantage
This system has been minimized by removing packages and content that are
not required on a system that users do not log into.

To restore this content, you can run the 'unminimize' command.

The programs included with the Ubuntu system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Ubuntu comes with ABSOLUTELY NO WARRANTY, to the extent permitted by
applicable law.

SUCCESS: on host: 10.44.227.112
ssh: (host: String, logStdout: Boolean)String

In [0]:
%sh
cat /root/.ssh/known_hosts >> /home/ubuntu/.ssh/known_hosts

In [0]:
%sh
chown -R ubuntu:ubuntu /home/ubuntu/.ssh/

In [0]:
%sh
#!/usr/bin/env bash
sudo -i -u ubuntu bash << EOF
whoami
/databricks/python/bin/python -V
. /databricks/conda/etc/profile.d/conda.sh
conda activate /databricks/python
mpiexec -n 3 -f /home/ubuntu/.ssh/hostfile -prepend-rank hostname
EOF

ubuntu
Python 3.8.8
[1] 1014-132551-ywrn3zez-10-44-235-163
[2] 1014-132551-ywrn3zez-10-44-227-112
[0] 1014-132551-ywrn3zez-10-44-233-230